# OPER 618 Game Theory and Math Programming
## Homework #3 - Computing Concepts for Normal Form Games
## Hosley, Brandon

In [28]:
import pyomo.environ as pyo
import itertools

#max(m.p.extract_values().values())
#max(m.p, key=m.p.extract_values().get)

1. **Maxmin Strategies**. Consider the two-player normal form game depicted in Figure 4.1 of
the textbook (and shown below), as well as the file *"OPER 618 - Homework 3 - Problem 1 -
Sets and Parameters.py"* posted on Canvas.

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
|              | $j_1$ |   0, 1   |  6, 0 |
| **Player 1** | $j_2$ |   2, 0   |  5, 2 |
|              | $j_3$ |   3, 4   |  3, 3 |

</div>

<ol type="a">
  <li>
    For this non-zero sum game, use Python/Pyomo to formulate and solve the problem of
    finding the <i>maxmin</i> strategy for <b>Player 1</b>, allowing for mixed strategies. Discuss the
    significance of the optimal solution
  </li>
</ol>


In [2]:
def problem1_a():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2,3}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
    [0, 0, 6], 
    [0, 2, 5],
    [0, 3, 3]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.p1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]

    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U1, sense = pyo.maximize)
    m.MincriteriaP1=pyo.ConstraintList()
    for k in m.K:
        m.MincriteriaP1.add(sum(u1[j][k]*m.p1[j] for j in m.J) >= m.U1)
    
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(m.p1[j] for j in m.J)==1)
                        
    # Solve model
    pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1))

    for j in m.J:
        print('p(',j,') = %.3f' % pyo.value(m.p1[j]))

problem1_a()

U1 = 3.000
p( 1 ) = 0.000
p( 2 ) = 0.000
p( 3 ) = 1.000


This solution shows that Player 1 will always choose $j_3$ under the maxmin strategy. Unsuprising, as they are gauranteed to have a utility no less than 3.

<ol type="a" start="2">
  <li>
    For this non-zero sum game, use Python/Pyomo to formulate and solve the problem of
    finding the <i>maxmin</i> strategy for <b>Player 2</b>, allowing for mixed strategies. Discuss the
    significance of the optimal solution.
  </li>
</ol>

In [3]:
def problem1_b():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2,3}
    m.K = {1,2}

    # Input parameter data
    u2=[[0,0,0],
    [0, 1, 0], 
    [0, 0, 2],
    [0, 4, 3]]

    # Decision Variables
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]

    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U2, sense = pyo.maximize)
    m.MincriteriaP2=pyo.ConstraintList()
    for j in m.J:
        m.MincriteriaP2.add(sum(u2[j][k]*m.p2[k] for k in m.K) >= m.U2)
    
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(m.p2[k] for k in m.K)==1)

    # Solve model
    pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U2 = %.3f' % pyo.value(m.U2)) 

    for k in m.K:
        print('p(',k,') = %.3f' % pyo.value(m.p2[k]))

problem1_b()

U2 = 0.667
p( 1 ) = 0.667
p( 2 ) = 0.333


Because both of Player 2's choices have equal minimums and neither is (weakly) dominant, a pure strategy does not exist.
As a result pyomo returns a solution in which the minimum non-zero expected results are equal. 

2. **Discriminating among Nash Equilibria with the LCP formulation**. Consider the two-player
normal form game depicted in Figure 4.1 of the textbook (and shown below), as well as the
file *"OPER 618 - Homework 3 - Problem 2 - LCP System of Equations.py"* posted on Canvas.

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
|              | $j_1$ |   0, 1   |  6, 0 |
| **Player 1** | $j_2$ |   2, 0   |  5, 2 |
|              | $j_3$ |   3, 4   |  3, 3 |

</div>

As formulated, the Python/Pyomo Concrete Model finds the Nash equilibrium $s = (s_1, s_2) = \left((0.67,0.33,0), (0.33, 0.67)\right)$ 
with expected payoffs of $(u_1, u_2) = (4, 0.67)$.
However, this is only one of the three Nash equilibria, and it doesn't seem to be a very
desirable one... at least not for Player 2.

<ol type="a">
  <li>
    Modify and solve the Python/Pyomo formulation to find the Nash equilibrium that
    <u>minimizes the maximum difference in the players' utilities</u>.
  </li>
</ol>

In [35]:
def problem2_a():
   m = pyo.ConcreteModel()

   # Input sets
   m.J = {1,2,3}
   m.K = {1,2}

   # Input parameter data
   u1=[[0,0,0],
      [0, 0, 6], 
      [0, 2, 5],
      [0, 3, 3]]
   u2=[[0,0,0],
      [0, 1, 0], 
      [0, 0, 2],
      [0, 4, 3]]

   # Decision Variables
   m.U1 = pyo.Var(domain=pyo.Reals) # Player 1's utility
   m.U2 = pyo.Var(domain=pyo.Reals) # Player 2's utility
   m.s1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # s1[j] is the probabilty that Player 1 plays action j
   m.r1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # Player 1's slack variable associated with the utility of playing action j 
   m.s2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # s2[k] is the probabilty that Player 2 plays action k
   m.r2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # Player 2's slack variable associated with the utility of playing action k 
   m.z = pyo.Var(domain=pyo.NonNegativeReals) # A dummy variable to serve as the objective function
   
   # Model (Formulation from page 91 of the text)
   m.objfnvalue = pyo.Objective(expr = m.z, sense = pyo.minimize)
   m.UtilityForPlayer1=pyo.ConstraintList()
   for j in m.J:
      m.UtilityForPlayer1.add(sum(u1[j][k]*m.s2[k] for k in m.K) + m.r1[j] >= m.z)
      #m.UtilityForPlayer1.add(sum(u1[j][k]*m.s2[k] for k in m.K) + m.r1[j] == m.U1)
   m.UtilityForPlayer2=pyo.ConstraintList()
   for k in m.K:
      m.UtilityForPlayer2.add(sum(u2[j][k]*m.s1[j] for j in m.J) + m.r2[k] >= m.z)
      #m.UtilityForPlayer2.add(sum(u2[j][k]*m.s1[j] for j in m.J) + m.r2[k] == m.U2)
   m.ProbabilityDistributionForP1 = pyo.Constraint(expr = sum(m.s1[j] for j in m.J) == 1)   
   m.ProbabilityDistributionForP2 = pyo.Constraint(expr = sum(m.s2[k] for k in m.K) == 1)                         
   m.ComplementarySlacknessForPlayer1=pyo.ConstraintList()
   for j in m.J:
      m.ComplementarySlacknessForPlayer1.add(m.s1[j]*m.r1[j] == 0)
   m.ComplementarySlacknessForPlayer2=pyo.ConstraintList()
   for k in m.K:
      m.ComplementarySlacknessForPlayer2.add(m.s2[k]*m.r2[k] == 0)

   # Solve model
   pyo.SolverFactory('ipopt').solve(m,tee=False)

   # Output results
   print('Difference = %.3f' % pyo.value(m.z))
   
   m.U1 = sum(u1[j][k]*m.s1[j] for k,j in list(itertools.product(m.K,m.J)))
   print('U1 = %.3f' % pyo.value(m.U1))
   for j in m.J:
      print('s1(',j,') = %.3f' % pyo.value(m.s1[j]))
      
   m.U2 = sum(u2[j][k]*m.s2[k] for k,j in list(itertools.product(m.K,m.J)))
   print('U2 = %.3f' % pyo.value(m.U2))    
   for k in m.K:
      print('s2(',k,') = %.3f' % pyo.value(m.s2[k]))

problem2_a()

Difference = -0.000
U1 = 6.247
s1( 1 ) = 0.219
s1( 2 ) = 0.247
s1( 3 ) = 0.534
U2 = 5.000
s2( 1 ) = 0.284
s2( 2 ) = 0.716


<ol type="a" start="2">
  <li>
    Modify and solve the Python/Pyomo formulation with an objective function that will
    find the remaining Nash equilibrium. Discuss the merits of your approach as it pertains
    to generalizability for other two-player games.
  </li>
</ol>

In [58]:
def problem2_b():
   m = pyo.ConcreteModel()

   # Input sets
   m.J = {1,2,3}
   m.K = {1,2}

   # Input parameter data
   u1=[[0,0,0],
      [0, 0, 6], 
      [0, 2, 5],
      [0, 3, 3]]
   u2=[[0,0,0],
      [0, 1, 0], 
      [0, 0, 2],
      [0, 4, 3]]

   # Decision Variables
   m.U1 = pyo.Var(domain=pyo.Reals) # Player 1's utility
   m.U2 = pyo.Var(domain=pyo.Reals) # Player 2's utility
   m.s1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # s1[j] is the probabilty that Player 1 plays action j
   m.r1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # Player 1's slack variable associated with the utility of playing action j 
   m.s2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # s2[k] is the probabilty that Player 2 plays action k
   m.r2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # Player 2's slack variable associated with the utility of playing action k 
   m.z = pyo.Var(domain=pyo.NonNegativeReals) # A dummy variable to serve as the objective function
   
   # Model (Formulation from page 91 of the text)
   #m.objfnvalue1 = pyo.Objective(expr = m.U1, sense = pyo.maximize)
   #m.objfnvalue2 = pyo.Objective(expr = m.U2, sense = pyo.minimize)
   m.objfnvalue = pyo.Objective(expr = m.U2, sense = pyo.minimize)

   m.UtilityForPlayer1=pyo.ConstraintList()
   for k in m.K:
      m.UtilityForPlayer1.add(sum(u1[j][k]*m.s1[j] for j in m.J) >= m.U1)

   m.UtilityForPlayer2=pyo.ConstraintList()
   for j in m.J:
      m.UtilityForPlayer2.add(sum(u2[j][k]*m.s2[k] for k in m.K) >= m.U2)

   m.ProbabilityDistributionForP1 = pyo.Constraint(expr = sum(m.s1[j] for j in m.J) == 1)   
   m.ProbabilityDistributionForP2 = pyo.Constraint(expr = sum(m.s2[k] for k in m.K) == 1)                         

   # Solve model
   results=pyo.SolverFactory('ipopt').solve(m,tee=False)

   # Output results
   print('U1 = %.3f' % pyo.value(m.U1)) 
   for j in m.J:
      print('s1(',j,') = %.3f' % pyo.value(m.s1[j]))
   print('U2 = %.3f' % pyo.value(m.U2))    
   for k in m.K:
      print('s2(',k,') = %.3f' % pyo.value(m.s2[k]))

problem2_b()

U1 = 3.000
s1( 1 ) = 0.667
s1( 2 ) = 0.333
s1( 3 ) = -0.000
U2 = 0.667
s2( 1 ) = 0.328
s2( 2 ) = 0.672


<ol type="a" start="3">
  <li>
    Discuss method one could implement to find <u>all</u> Nash equilibria using the LCP formulation.
  </li>
</ol>

### Normal form game used for Problems 3, 4, & 5. 
Consider the two-player normal form game depicted below, which is a modified form of "Chicken".

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
| **Player 1** | $j_1$ |   0, 0   |  9, 3 |
|              | $j_2$ |   3, 9   |  7, 7 |

</div>
The following figure (Duffy and Feltovich, 2010) shows the possible payoffs for Player 1 and
Player 2, wherein the respective regions of feasible payoffs, correlated equilibrium payoffs,
and Nash equilibrium payoffs are depicted with the indicated letter(s).

![](https://www.mathworks.com/matlabcentral/mlc-downloads/downloads/e568cfd1-4a80-11e4-9553-005056977bd0/66a2877e-5aac-4a1d-b090-5035e35db9b2/images/screenshot.gif)

3. **Correlated Equilibria and Nash Equilibria**. Consider the Python/Pyomo models in the following files, which correspond to the two functional representations in Section 4.6.
    - *OPER 618 - Homework 3 - Problem 3 - Correlated Equilibrium (LP) - Chicken.py*
    - *OPER 618 - Homework 3 - Problem 3 - Nash Equilibrium (NLP) - Chicken.py*

    Modify the objective function(s) and resolve the model(s) as necessary to identify (a) the
    four correlated equilibria corresponding to the extreme points of the region of correlated
    equilibria payoff pairs from the diagram and (b) the three correlated equilibria
    corresponding to the three Nash equilibria corresponding to the extreme points of the
    region of NE payoff pairs from the diagram.

4. **The Region of Correlated Equilibria**. Identify a correlated equilibrium yielding a pair of
payoffs in the region $B_l$ (i.e., the region of correlated equilibria that are not Nash equilibria
and have lower-than-NE payoffs), as well as a correlated equilibrium yielding a pair of
payoffs in the region $B_h$.

5. **Unstable Solutions**. Show that the solution corresponding to the joint payoffs of $(7,7)$ is
neither a correlated equilibrium nor a Nash equilibrium.